In [1]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading

logging.basicConfig(filename='trading_GBPUSD_Buy_Sell_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

def fetch_latest_candle(symbol, timeframe):
    latest_candles = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1)
    return latest_candles[0] if len(latest_candles) > 0 else None

def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

def add_rolling_features(df, window):
    logging.info("Adding rolling features")
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

def add_lag_features(df, lags):
    logging.info("Adding lag features")
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

def calculate_indicators(df):
    logging.info("Calculating indicators")
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

def align_features_with_json(df, features_json):
    expected_features = features_json
    missing_features = [feat for feat in expected_features if feat not in df.columns]
    extra_features = [feat for feat in df.columns if feat not in expected_features]
    
    logging.info(f"Missing features: {missing_features}")
    logging.info(f"Extra features: {extra_features}")
    
    for feat in missing_features:
        df[feat] = 0

    df = df[expected_features]
    
    return df

def process_data_for_features(df, symbol, signals, selected_features):
    logging.info(f"Processing data for feature extraction for {symbol}")
    X_combined = pd.DataFrame(index=df['time'])
    
    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time", max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]
        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    
    logging.info(f"NaNs before drop for {symbol}: {df.isnull().sum().sum()}")
    df = df.dropna().reset_index(drop=True)
    logging.info(f"NaNs after drop for {symbol}: {df.isnull().sum().sum()}")

    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')
    combined_df = combined_df[selected_features]
    logging.info(f"NaNs after merging for {symbol}: {combined_df.isnull().sum().sum()}")
    combined_df = combined_df.dropna()
    logging.info(f"Final NaNs for {symbol}: {combined_df.isnull().sum().sum()}")
    
    return combined_df

def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size):
    logging.info("Calculating SL and TP prices")
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    sl_price = entry_price - risk_amount
    tp_price = entry_price + reward_amount
    return sl_price, tp_price

def place_order(symbol, volume, sl_price, tp_price, trade_type):
    logging.info(f"Placing {trade_type} order for {symbol}")
    
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None or not symbol_info.visible or symbol_info.trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        logging.error(f"Market for {symbol} is closed or unavailable.")
        print(f"Market for {symbol} is closed. Cannot place order.")
        return

    price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if trade_type == "Buy" else mt5.ORDER_TYPE_SELL
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": order_type,
        "price": price,
        "sl": sl_price,
        "tp": tp_price,
        "deviation": 10,
        "magic": 0,
        "comment": f"Python script {trade_type} order",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(request)
    
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        logging.info(f"{trade_type} order placed successfully for {symbol}.")
        print(f"{trade_type} order placed successfully for {symbol}.")
    else:
        logging.error(f"Order placement failed for {symbol}: {result.retcode} - {mt5.last_error()}")
        print(f"Order placement failed for {symbol}: {result.retcode} - {mt5.last_error()}")

def apply_custom_threshold(probas, custom_threshold):
    return (probas >= custom_threshold).astype(int)

def predict_and_trade(scaler, model, symbol, volume, timeframe, risk_reward_ratio, mean_candle_size, start_date, end_date, features_json, trade_type, threshold=None):
    logging.info(f"Starting prediction and trading process for {symbol} - {trade_type}")
    signals = ['WILLR_15', 'WILLR_42']
    df_processed = fetch_and_process_data(symbol, timeframe, signals, features_json, start_date, end_date)
    original_index = df_processed.index
    df_processed = df_processed.shift(periods=1, axis=0).dropna()
    df_processed.index = original_index
    df_processed = align_features_with_json(df_processed, features_json)
    logging.info(f"Number of features for {symbol} - {trade_type}: {df_processed.shape[1]}")
    scaled_data = scaler.transform(df_processed)
    probas = model.predict_proba(scaled_data)[:, 1]
    custom_threshold = threshold if threshold is not None else 0.5
    y_pred = apply_custom_threshold(probas, custom_threshold)
    
    df_pred = pd.DataFrame(index=df_processed.index)
    df_pred['prediction'] = y_pred
    save_predictions_to_csv(df_pred, symbol, trade_type)
    
    if len(y_pred) > 1 and y_pred[-1] == 1:
        entry_price = mt5.symbol_info_tick(symbol).ask if trade_type == "Buy" else mt5.symbol_info_tick(symbol).bid
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size)
        place_order(symbol, volume, sl_price, tp_price, trade_type)
    else:
        logging.info(f"No {trade_type} signal generated for {symbol}. Doing nothing.")
        print(f"No {trade_type} signal generated for {symbol}. Doing nothing.")

def save_predictions_to_csv(df_pred, symbol, trade_type):
    logging.info(f"Saving {trade_type} predictions to CSV for {symbol}")
    file_path = f'predict_{symbol}_D1_3112_{trade_type}.csv'
    if not os.path.isfile(file_path):
        df_pred.to_csv(file_path, index=True)
    else:
        df_pred.to_csv(file_path, mode='a', header=False, index=True)

def process_pair(config, utc_from, utc_to):
    logging.info(f"Processing pair: {config['symbol']} for Buy and Sell")
    buyscaler, buymodel, sellscaler, sellmodel = None, None, None, None
    try:
        buyscaler = load(config["buy_scaler_path"])
        buymodel = load(config["buy_model_path"])
    except Exception as e:
        logging.warning(f"Buy model or scaler missing for {config['symbol']}: {e}")
        print(f"Buy model or scaler missing for {config['symbol']}: {e}")
    try:
        sellscaler = load(config["sell_scaler_path"])
        sellmodel = load(config["sell_model_path"])
    except Exception as e:
        logging.warning(f"Sell model or scaler missing for {config['symbol']}: {e}")
        print(f"Sell model or scaler missing for {config['symbol']}: {e}")
    
    with open(config["features_path"], 'r') as f:
        feature_names = json.load(f)
    buy_features = feature_names[:-1]
    sell_features = feature_names[:-1]

    last_candle = fetch_latest_candle(config["symbol"], config["timeframe"])
    if last_candle is None:
        logging.error(f"Failed to fetch initial candle data for {config['symbol']}.")
        print(f"Failed to fetch initial candle data for {config['symbol']}.")
        return

    try:
        while True:
            current_candle = fetch_latest_candle(config["symbol"], config["timeframe"])
            if current_candle and current_candle['open'] != last_candle['open']:
                if buyscaler is not None and buymodel is not None:
                    predict_and_trade(
                        buyscaler, buymodel, config["symbol"], config["volume"], config["timeframe"],
                        config["buy_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, buy_features, "Buy",
                        threshold=config.get("buy_threshold", None)
                    )
                if sellscaler is not None and sellmodel is not None:
                    predict_and_trade(
                        sellscaler, sellmodel, config["symbol"], config["volume"], config["timeframe"],
                        config["sell_risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, sell_features, "Sell",
                        threshold=config.get("sell_threshold", None)
                    )
                last_candle = current_candle
            time.sleep(60)
    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")

def main():
    config = {'login': 51708234, 'password': '4bM&wuVJcBTnjV', 'server': 'ICMarketsEU-Demo'}
    init_mt5_connection(config['login'], config['password'], config['server'])
    
    gbpusd_config = {
        "symbol": "GBPUSD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.013,
        "buy_risk_reward_ratio": "1:1",
        "buy_scaler_path": 'GBPUSD_D1_3112_NEWBuy/scaler_fold_1.joblib',
        "buy_model_path": 'GBPUSD_D1_3112_NEWBuy/best_model_fold_1.joblib',
        "sell_scaler_path": 'GBPUSD_D1_3112_Sell/scaler_fold_1.joblib',
        "sell_model_path": 'GBPUSD_D1_3112_Sell/best_model_fold_1.joblib',
        "features_path": 'GBPUSD_D1_3112_NEWBuy/feature_names.json',
        "buy_threshold": 0.55
    }

    utc_from = datetime(2023, 5, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    process_pair(gbpusd_config, utc_from, utc_to)

if __name__ == "__main__":
    main()


c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


Connected to MetaTrader 5
Sell model or scaler missing for GBPUSD: [Errno 2] No such file or directory: 'GBPUSD_D1_3112_Sell/scaler_fold_1.joblib'


c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\tsfresh\utilities\dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Feature Extraction: 100%|██████████| 35/35 [00:04<00:00,  7.23it/s]
c:\ProgramData\anaconda3\envs\forex_env\Lib\site-packages\tsfresh\utilities\dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Feature Extraction: 100%|██████████| 35/35 [00:08<00:00,  4.27it/s]


MetaTrader 5 connection closed for GBPUSD.


ValueError: Length mismatch: Expected axis has 206 elements, new values have 207 elements

In [ ]:
import json
import sys
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
import pytz
import time
from joblib import load
import logging
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
import talib
import os
import threading

logging.basicConfig(filename='trading_GBPUSDCAD_Buy_D1.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to load configuration (adjusted for direct input in notebook)
def load_config(config_path):
    with open(config_path, 'r') as config_file:
        return json.load(config_file)

# Initialize connection to MetaTrader 5
def init_mt5_connection(login, password, server):
    if not mt5.initialize(login=login, password=password, server=server):
        logging.error(f"initialize() failed, error code = {mt5.last_error()}")
        sys.exit()
    logging.info("Connected to MetaTrader 5")
    print("Connected to MetaTrader 5")

# Fetch the latest candle
def fetch_latest_candle(symbol, timeframe):
    latest_candles = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1)
    return latest_candles[0] if len(latest_candles) > 0 else None

# Fetch historical data
def fetch_historical_data(symbol, timeframe, start_date, end_date):
    logging.info(f"Fetching historical data for {symbol}")
    print(f"Fetching historical data for {symbol}")
    data = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    ohlc_data = pd.DataFrame(data)
    ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')
    return ohlc_data[['time', 'open', 'high', 'low', 'close']]

def add_rolling_features(df, window):
    logging.info("Adding rolling features")
    df['rolling_mean_open'] = df['open'].rolling(window=window).mean()
    df['rolling_std_open'] = df['open'].rolling(window=window).std()
    df['rolling_mean_close'] = df['close'].rolling(window=window).mean()
    df['rolling_std_close'] = df['close'].rolling(window=window).std()
    df['rolling_mean_high'] = df['high'].rolling(window=window).mean()
    df['rolling_std_high'] = df['high'].rolling(window=window).std()
    df['rolling_mean_low'] = df['low'].rolling(window=window).mean()
    df['rolling_std_low'] = df['low'].rolling(window=window).std()
    return df

# Function to add lag features
def add_lag_features(df, lags):
    logging.info("Adding lag features")
    for lag in lags:
        df[f'open_lag_{lag}'] = df['open'].shift(lag)
        df[f'close_lag_{lag}'] = df['close'].shift(lag)
        df[f'high_lag_{lag}'] = df['high'].shift(lag)
        df[f'low_lag_{lag}'] = df['low'].shift(lag)
    return df

# Calculate indicators
def calculate_indicators(df):
    logging.info("Calculating indicators")
    for period in [15, 23, 42, 145]:
        df[f'WILLR_{period}'] = talib.WILLR(df['high'], df['low'], df['close'], timeperiod=period)
    return df

# Process data for feature extraction
def process_data_for_features(df, symbol, signals, selected_features):
    logging.info(f"Processing data for feature extraction for {symbol}")
    X_combined = pd.DataFrame(index=df['time'])
    
    for signal in signals:
        df_melted = df[['time', signal]].copy()
        df_melted["Symbols"] = symbol
        df_rolled = roll_time_series(df_melted, column_id="Symbols", column_sort="time",
                                     max_timeshift=20, min_timeshift=5)
        X = extract_features(df_rolled.drop("Symbols", axis=1), column_id="id", column_sort="time", 
                             column_value=signal, impute_function=impute, show_warnings=False)
        X = X.set_index(X.index.map(lambda x: x[1]), drop=True)
        
        # Filter features
        selected_features_X = [feature for feature in selected_features if feature in X.columns]
        X_filtered = X[selected_features_X]
        
        X_combined = X_combined.merge(X_filtered, left_index=True, right_index=True, how='left')
        X_combined = X_combined.dropna()
    return X_combined

# Combine buy data
def fetch_and_process_data(symbol, timeframe, signals, selected_features, start_date, end_date):
    logging.info(f"Fetching and processing data for {symbol}")
    print(f"Fetching and processing data for {symbol}")
    df = fetch_historical_data(symbol, timeframe, start_date, end_date)
    df = calculate_indicators(df)
    df = add_rolling_features(df, window=5)
    df = add_lag_features(df, lags=[1, 2, 3, 4, 5])
    
    # Log the number of NaNs before dropping
    logging.info(f"NaNs before drop for {symbol}: {df.isnull().sum().sum()}")

    df = df.dropna().reset_index(drop=True)
    
    # Log the state after dropping NaNs
    logging.info(f"NaNs after drop for {symbol}: {df.isnull().sum().sum()}")

    features = process_data_for_features(df, symbol, signals, selected_features)
    df = df.set_index('time')
    combined_df = df.merge(features, left_index=True, right_index=True, how='left')
    
    # Handle missing values in the combined dataset
    combined_df = combined_df[selected_features]
    
    # Log the state of NaNs after merging
    logging.info(f"NaNs after merging for {symbol}: {combined_df.isnull().sum().sum()}")
    
    combined_df = combined_df.dropna()
    
    # Log final NaN count
    logging.info(f"Final NaNs for {symbol}: {combined_df.isnull().sum().sum()}")
    
    return combined_df

# Calculate SL and TP based on entry price and specified percentages
def calculate_prices(entry_price, risk_reward_ratio, mean_candle_size):
    logging.info("Calculating SL and TP prices")
    risk_part, reward_part = map(int, risk_reward_ratio.split(':'))
    risk_amount = mean_candle_size * risk_part
    reward_amount = mean_candle_size * reward_part
    sl_price = entry_price - risk_amount
    tp_price = entry_price + reward_amount
    return sl_price, tp_price

# Place order
def place_order(symbol, volume, sl_price, tp_price):
    logging.info(f"Placing order for {symbol}")
    
    # Check if the market is open for the symbol
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        logging.error(f"Failed to get symbol info for {symbol}.")
        return
    
    if not symbol_info.visible or symbol_info.trade_mode == mt5.SYMBOL_TRADE_MODE_DISABLED:
        logging.error(f"Market for {symbol} is currently closed or not available for trading.")
        print(f"Market for {symbol} is closed. Cannot place order.")
        return

    # Proceed with order placement
    price = mt5.symbol_info_tick(symbol).ask
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": sl_price,
        "tp": tp_price,
        "deviation": 10,
        "magic": 0,
        "comment": "Python script open",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    result = mt5.order_send(request)
    
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        logging.info(f"Order placed successfully for {symbol}.")
        print(f"Order placed successfully for {symbol}.")
    elif result.retcode == 10018:  # Market closed
        logging.error(f"Market closed for {symbol}. Order not placed.")
        print(f"Market closed for {symbol}. Order not placed.")
    else:
        logging.error(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")
        print(f"Order placement failed for {symbol}: {result.retcode if result else 'No response'} - {mt5.last_error()}")


# Function to apply the custom threshold
def apply_custom_threshold(probas, custom_threshold):
    return (probas >= custom_threshold).astype(int)

# Predict and trade
def predict_and_trade(buyscaler, buymodel, symbol, volume, timeframe, risk_reward_ratio, mean_candle_size, start_date, end_date, buy_features):
    logging.info(f"Starting prediction and trading process for {symbol}")
    print(f"Starting prediction and trading process for {symbol}")
    buy_signals = ['WILLR_15', 'WILLR_42']
    
    # Fetch and process data
    df_buy = fetch_and_process_data(symbol, timeframe, buy_signals, buy_features, start_date, end_date)

    # Shift the data as was done during training
    logging.info(f"Shifting the data for {symbol}")
    original_index = df_buy.index
    shifted_df_buy = df_buy.shift(periods=1, axis=0)
    shifted_df_buy.index = original_index
    df_buy = shifted_df_buy.dropna()  # Drop NaNs after shifting

    # Align the features with what the scaler expects
    df_buy = df_buy[buy_features]

    # Scale the data
    logging.info(f"Scaling the data for {symbol}")
    scaled_buy = buyscaler.transform(df_buy)
    
    # Predict probabilities and apply custom threshold
    logging.info(f"Predicting for {symbol}")
    probas_buy = buymodel.predict_proba(scaled_buy)[:, 1]
    custom_threshold = 0.5  # If you have a specific threshold, set it here
    y_pred = apply_custom_threshold(probas_buy, custom_threshold)
    
    # Create a DataFrame to hold prediction data with index matching test data
    df_pred = pd.DataFrame(index=df_buy.index)
    df_pred['prediction'] = y_pred
    
    # Save predictions to CSV
    save_predictions_to_csv(df_pred, symbol)
    
    if len(y_pred) > 1 and y_pred[-1] == 1:  # If the second-last prediction is a buy signal
        entry_price = mt5.symbol_info_tick(symbol).ask
        sl_price, tp_price = calculate_prices(entry_price, risk_reward_ratio, mean_candle_size)
        place_order(symbol, volume, sl_price, tp_price)
    else:
        logging.info(f"No Buy signal generated for {symbol}. Doing nothing.")
        print(f"No Buy signal generated for {symbol}. Doing nothing.")


def save_predictions_to_csv(df_pred, symbol):
    logging.info(f"Saving predictions to CSV for {symbol}")
    file_path = f'predict_{symbol}_D1_3112_Buy.csv'
    if not os.path.isfile(file_path):
        df_pred.to_csv(file_path, index=True)
    else:
        df_pred.to_csv(file_path, mode='a', header=False, index=True)

# Function to process each pair in a separate thread
def process_pair(config, utc_from, utc_to):
    logging.info(f"Processing pair: {config['symbol']}")
    print(f"Processing pair: {config['symbol']}")
    
    buyscaler = load(config["scaler_path"])
    buymodel = load(config["model_path"])
    with open(config["features_path"], 'r') as f:
        feature_names = json.load(f)
    buy_features = feature_names[:-1]  # Exclude the last feature from the list

    last_candle = fetch_latest_candle(config["symbol"], config["timeframe"])
    if last_candle is None:
        logging.error(f"Failed to fetch initial candle data for {config['symbol']}.")
        print(f"Failed to fetch initial candle data for {config['symbol']}.")
        return

    try:
        while True:
            current_candle = fetch_latest_candle(config["symbol"], config["timeframe"])
            if current_candle and current_candle['open'] != last_candle['open']:
                predict_and_trade(
                    buyscaler, buymodel, config["symbol"], config["volume"], config["timeframe"],
                    config["risk_reward_ratio"], config["mean_candle_size"], utc_from, utc_to, buy_features
                )
                last_candle = current_candle
            time.sleep(60)
    except KeyboardInterrupt:
        logging.info(f"Script terminated by user for {config['symbol']}.")
        print(f"Script terminated by user for {config['symbol']}.")
    finally:
        mt5.shutdown()
        logging.info(f"MetaTrader 5 connection closed for {config['symbol']}.")
        print(f"MetaTrader 5 connection closed for {config['symbol']}.")

# Main function
def main():
    # Sample configuration, replace with your actual configuration details
    config = {'login': 51708234, 'password': '4bM&wuVJcBTnjV', 'server': 'ICMarketsEU-Demo'}

    init_mt5_connection(config['login'], config['password'], config['server'])
    
    # Configuration for GBPUSD
    gbpusd_config = {
        "symbol": "GBPUSD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.013,
        "risk_reward_ratio": "1:1",
        "scaler_path": 'GBPUSD_D1_3112_NEWBuy/scaler_fold_1.joblib',
        "model_path": 'GBPUSD_D1_3112_NEWBuy/best_model_fold_1.joblib',
        "features_path": 'GBPUSD_D1_3112_NEWBuy/feature_names.json'
    }

    # Configuration for USDCAD
    usdcad_config = {
        "symbol": "USDCAD",
        "timeframe": mt5.TIMEFRAME_D1,
        "volume": 0.1,
        "mean_candle_size": 0.088,
        "risk_reward_ratio": "2:3",
        "scaler_path": 'USDCAD_D1_3112_BuyNEW/scaler.joblib',
        "model_path": 'USDCAD_D1_3112_BuyNEW/model.joblib',
        "features_path": 'USDCAD_D1_3112_BuyNEW/feature_names.json' 
    }

    # Load the configurations
    configs = [gbpusd_config, usdcad_config]

    utc_from = datetime(2023, 5, 1, tzinfo=pytz.utc)
    utc_to = datetime.now(pytz.utc)

    threads = []
    for config in configs:
        thread = threading.Thread(target=process_pair, args=(config, utc_from, utc_to))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()
